# annotate images for U-Net training

In [1]:
import os
import numpy as np
import napari

In [2]:
from skimage.io import imsave

In [3]:
PATH = "/Users/arl/Downloads/images_to_annotate"
GT_PATH = os.path.join(PATH, "GT")

In [4]:
from daskoctopus import DaskOctopusLiteLoader

In [5]:
reader = DaskOctopusLiteLoader(
    PATH, 
    crop=(1200, 1600),
    remove_background=True,
)

Using cropping: (1200, 1600)


In [6]:
brightfield = reader["brightfield"]
files = reader.files("brightfield")

In [7]:
viewer = napari.Viewer()
viewer.add_image(brightfield)
viewer.add_labels(np.zeros(brightfield.shape, dtype=np.uint64), name="ground_truth")

@viewer.bind_key('x')
def save_ground_truth(viewer):

    idx = viewer.dims.current_step[0]
    _, raw_fn = os.path.split(files[idx])
    fn = os.path.join(GT_PATH, f"GT_{raw_fn}")

    if not os.path.exists(GT_PATH):
        os.makedirs(GT_PATH)
    
    print(idx, fn)
    gt = viewer.layers["ground_truth"].data[idx, ...]
    imsave(fn, gt.astype(np.uint8))